In [1]:
import pandas as pd

conditions_2 = pd.read_csv("/kaggle/input/conditions-2/conditions-2.csv")
conditions_1 = pd.read_csv("/kaggle/input/conditions-1/conditions-1.csv")
conditions = pd.concat([conditions_2, conditions_1])
conditions = conditions[['PATIENT', 'ENCOUNTER', 'DESCRIPTION']]
patients_2 = pd.read_csv("/kaggle/input/patients-2/patients-2.csv")
patients_1 = pd.read_csv("/kaggle/input/patients-1/patients-1.csv")
patients = pd.concat([patients_1, patients_2])
patients = patients[['Id', 'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'RACE', 'ETHNICITY', 'GENDER', 'ADDRESS', 'ZIP']]
medications_2 = pd.read_csv("/kaggle/input/medication-2/medications-2.csv")
medications_1 = pd.read_csv("/kaggle/input/medication-1/medications-1.csv")
medications = pd.concat([medications_2, medications_1])
medications = medications[['START', 'PATIENT', 'ENCOUNTER']]

In [2]:
medications['START'] = pd.to_datetime(medications['START'])
medications = medications[(medications['START'].dt.year == 2019) | (medications['START'].dt.year == 2020)]
patients_19_20 = list(medications['PATIENT'].unique())

patients = patients[patients['Id'].isin(patients_19_20)]
conditions = conditions[conditions['PATIENT'].isin(patients_19_20)]

del patients_19_20

combined_1 = conditions.merge(patients, left_on = 'PATIENT', right_on = 'Id', how = 'left')
del patients
del conditions
combined = combined_1.merge(medications, on = 'ENCOUNTER', how = 'left')
combined_19_20 = combined[(combined['START'].dt.year == 2019) | (combined['START'].dt.year == 2020)]
del combined

In [3]:
Risk_desc_map = {'Arthritis Risk' : ['Chronic pain', 'Osteoarthritis of hip', 'Osteoarthritis of knee', 'Osteoporosis (disorder)', 'Pathological fracture due to osteoporosis (disorder)', 'Rheumatoid arthritis'], 
                 'Alcohol/Drug Abuse':['Reports of violence in the environment (finding)','Alcoholism', 'Smokes tobacco daily', 'Unhealthy alcohol drinking behavior (finding)', 'Drug overdose'], 
                 'Air Quality': ['Perennial allergic rhinitis', 'Perennial allergic rhinitis with seasonal variation'], 
                 'Lung Diseases' : ['Childhood asthma', 'Acute bacterial sinusitis (disorder)', 'Acute bronchitis (disorder)', 'Acute pulmonary embolism (disorder)', 'Acute respiratory distress syndrome (disorder)', 'Acute respiratory failure (disorder)', 'Acute viral pharyngitis (disorder)', 'Allergy to latex (finding)', 'Asthma', 'Chronic obstructive bronchitis (disorder)', 'Pneumonia', 'Pneumonia (disorder)', 'Respiratory distress (finding)', 'Sinusitis (disorder)', 'Small cell carcinoma of lung (disorder)', 'Sore throat symptom (finding)', 'Streptococcal sore throat (disorder)', 'Suspected COVID-19', 'Suspected lung cancer (situation)', 'Viral sinusitis (disorder)', 'Wheezing (finding)', 'Dyspnea (finding)'],
                 'Heart Diseases': ['Cardiac Arrest', 'Chronic congestive heart failure (disorder)', 'Stroke', 'Acute deep venous thrombosis (disorder)', 'Atrial Fibrillation'],
                 'Physical Health' : ['Body mass index 30+ - obesity (finding)', 'Body mass index 40+ - severely obese (finding)', 'Muscle pain (finding)', 'Poor muscle tone (finding)'],
                 'Mental Health': ['At risk for suicide (finding)', 'Posttraumatic stress disorder', 'Severe anxiety (panic) (finding)', 'Stress (finding)', 'Major depression  single episode', 'Attempted suicide - cut/stab'],
                 'Personality Disorder': ['Intellectual disability (disorder)'],
                 'Housing Challenges': ['Homeless (finding)', 'Housing unsatisfactory (finding)', 'Refugee (person)'],
                 'Transportation Risk': ['Lack of access to transportation (finding)', 'Transport problems (finding)'],
                 'Social Associations': ['Social isolation (finding)', 'Social migrant (finding)']
                }

In [5]:
for risk, descriptions in Risk_desc_map.items():
    combined_19_20[risk] = combined_19_20['DESCRIPTION'].isin(descriptions).astype(int)

In [8]:
combined_19_20 = combined_19_20.reset_index(drop = True)

In [9]:
combined_19_20

,PATIENT_x,ENCOUNTER,DESCRIPTION,Id,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,RACE,...,Alcohol/Drug Abuse,Air Quality,Lung Diseases,Heart Diseases,Physical Health,Mental Health,Personality Disorder,Housing Challenges,Transportation Risk,Social Associations
0,3bcec4db-d2fd-bd0b-9922-a8b72cb8cfae,fc5f977f-d45e-6c5f-07fd-e86a696bdf68,Viral sinusitis (disorder),3bcec4db-d2fd-bd0b-9922-a8b72cb8cfae,Mr.,Davis923,Hudson301,NaN,NaN,white,...,0,0,1,0,0,0,0,0,0,0
1,0cf4e57d-69e0-2e98-0f57-8d5e50afd45d,61d7e5e8-587c-afe1-f9c1-93c9fbc42700,Not in labor force (finding),0cf4e57d-69e0-2e98-0f57-8d5e50afd45d,Mrs.,Lilia791,Terán294,NaN,Banda20,white,...,0,0,0,0,0,0,0,0,0,0
2,0cf4e57d-69e0-2e98-0f57-8d5e50afd45d,61d7e5e8-587c-afe1-f9c1-93c9fbc42700,Not in labor force (finding),0cf4e57d-69e0-2e98-0f57-8d5e50afd45d,Mrs.,Lilia791,Terán294,NaN,Banda20,white,...,0,0,0,0,0,0,0,0,0,0
3,0cf4e57d-69e0-2e98-0f57-8d5e50afd45d,61d7e5e8-587c-afe1-f9c1-93c9fbc42700,Severe anxiety (panic) (finding,0cf4e57d-69e0-2e98-0f57-8d5e50afd45d,Mrs.,Lilia791,Terán294,NaN,Banda20,white,...,0,0,0,0,0,0,0,0,0,0
4,0cf4e57d-69e0-2e98-0f57-8d5e50afd45d,61d7e5e8-587c-afe1-f9c1-93c9fbc42700,Severe anxiety (panic) (finding,0cf4e57d-69e0-2e98-0f57-8d5e50afd45d,Mrs.,Lilia791,Terán294,NaN,Banda20,white,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209068,d03e5f55-ae57-f451-5a0f-76ccf432f3cf,40f43266-e9be-fcd9-763e-f5a6fe2f0d36,Full-time employment (finding),d03e5f55-ae57-f451-5a0f-76ccf432f3cf,Ms.,Sanda877,Pacocha935,NaN,NaN,black,...,0,0,0,0,0,0,0,0,0,0
209069,d03e5f55-ae57-f451-5a0f-76ccf432f3cf,bbbe7a97-65c3-c4d8-7088-c4d4adbda4c2,Part-time employment (finding),d03e5f55-ae57-f451-5a0f-76ccf432f3cf,Ms.,Sanda877,Pacocha935,NaN,NaN,black,...,0,0,0,0,0,0,0,0,0,0
209070,d03e5f55-ae57-f451-5a0f-76ccf432f3cf,bbbe7a97-65c3-c4d8-7088-c4d4adbda4c2,Part-time employment (finding),d03e5f55-ae57-f451-5a0f-76ccf432f3cf,Ms.,Sanda877,Pacocha935,NaN,NaN,black,...,0,0,0,0,0,0,0,0,0,0
209071,d03e5f55-ae57-f451-5a0f-76ccf432f3cf,bbbe7a97-65c3-c4d8-7088-c4d4adbda4c2,Limited social contact (finding),d03e5f55-ae57-f451-5a0f-76ccf432f3cf,Ms.,Sanda877,Pacocha935,NaN,NaN,black,...,0,0,0,0,0,0,0,0,0,0


In [14]:
len(combined_19_20.ZIP.unique())

328

In [23]:
combined_19_20.columns

Index(['PATIENT_x', 'ENCOUNTER', 'DESCRIPTION', 'Id', 'PREFIX', 'FIRST',
       'LAST', 'SUFFIX', 'MAIDEN', 'RACE', 'ETHNICITY', 'GENDER', 'ADDRESS',
       'ZIP', 'START', 'PATIENT_y', 'Arthritis Risk', 'Alcohol/Drug Abuse',
       'Air Quality', 'Lung Diseases', 'Heart Diseases', 'Physical Health',
       'Mental Health', 'Personality Disorder', 'Housing Challenges',
       'Transportation Risk', 'Social Associations'],
      dtype='object')

In [24]:
mode_columns = ['PATIENT_x', 'ENCOUNTER', 'DESCRIPTION', 'Id', 'PREFIX', 'FIRST',
       'LAST', 'SUFFIX', 'MAIDEN', 'RACE', 'ETHNICITY', 'GENDER', 'ADDRESS',
       'ZIP', 'START', 'PATIENT_y']
sum_columns = ['Arthritis Risk', 'Alcohol/Drug Abuse',
       'Air Quality', 'Lung Diseases', 'Heart Diseases', 'Physical Health',
       'Mental Health', 'Personality Disorder', 'Housing Challenges',
       'Transportation Risk', 'Social Associations']

In [28]:
aggregated_df = combined_19_20.groupby('ZIP').agg({
    **{col: lambda x: x.mode().iloc[0] if not x.mode().empty else None for col in mode_columns},
    **{col: 'sum' for col in sum_columns}
})

common_df = aggregated_df.reset_index(drop = True)

In [29]:
common_df

,PATIENT_x,ENCOUNTER,DESCRIPTION,Id,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,RACE,...,Alcohol/Drug Abuse,Air Quality,Lung Diseases,Heart Diseases,Physical Health,Mental Health,Personality Disorder,Housing Challenges,Transportation Risk,Social Associations
0,aa56ac89-c8e3-aa69-6518-d8c35e818241,fd0b9ab9-e6ee-a288-c898-8fa01588edbe,Full-time employment (finding),aa56ac89-c8e3-aa69-6518-d8c35e818241,Mrs.,Lore422,Leannon79,None,Littel644,white,...,6,0,3,0,0,17,0,2,0,0
1,3d5d20bd-e1ea-ffb6-99c3-3136a3db8e16,1cda6c5b-e3d6-c0cd-7212-cfd4c8d02b18,Full-time employment (finding),3d5d20bd-e1ea-ffb6-99c3-3136a3db8e16,Mr.,Wilfredo622,Bernhard322,None,Metz686,white,...,0,0,0,0,0,8,0,0,0,0
2,42b97dff-a290-d52b-6918-8a566486998f,9e4b71a0-8311-abcd-aad6-00ab19b84528,Full-time employment (finding),42b97dff-a290-d52b-6918-8a566486998f,Mr.,Jamaal34,VonRueden376,None,Brekke496,white,...,3,0,3,0,0,17,0,0,0,0
3,a8850406-cb1c-914a-27a1-b46ca149a14b,8eb657fa-e302-114a-8798-4f40d30985f5,Full-time employment (finding),a8850406-cb1c-914a-27a1-b46ca149a14b,Mrs.,Daria61,Brakus656,None,Strosin214,white,...,0,0,1,0,0,3,0,0,0,0
4,46d3dd4f-9a71-cd46-f019-f479b4366033,5f46251b-b8cf-5f7b-1f7e-cc0ac354d02d,Full-time employment (finding),46d3dd4f-9a71-cd46-f019-f479b4366033,Mrs.,Susie661,Crist667,None,Lueilwitz711,white,...,14,0,26,2,4,89,0,0,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,f360849e-af19-4392-88cb-5dbd5be15594,359bf0a3-4e0f-6522-98ab-113632918c36,Full-time employment (finding),f360849e-af19-4392-88cb-5dbd5be15594,Mr.,Daren950,Pacocha935,MD,Hermann103,white,...,7,0,11,0,8,42,0,6,0,1
323,8f575107-aff3-7a46-4db9-f1ef60b43544,aa1654c8-8fbe-5241-9ee8-80b1f9f684c5,Full-time employment (finding),8f575107-aff3-7a46-4db9-f1ef60b43544,Mr.,Adolph80,Bechtelar572,None,Koelpin146,white,...,2,0,4,0,0,8,0,0,0,0
324,732f91db-2bda-f472-8820-08ec88358b6f,d9b500c3-becf-ca7d-c753-ffce0eedc620,Full-time employment (finding),732f91db-2bda-f472-8820-08ec88358b6f,Mr.,Lou594,Dooley940,MD,Howell947,white,...,6,0,27,0,0,66,0,1,0,8
325,56f18b81-b5dd-9c17-d4b3-28b844877faa,290774b8-793d-b47b-f3bb-cdec1d57d10d,Full-time employment (finding),56f18b81-b5dd-9c17-d4b3-28b844877faa,Mr.,Saul605,Abshire638,PhD,Herman763,white,...,14,0,36,0,1,79,0,0,0,28


In [30]:
common_df = common_df.drop(columns = ['Unnamed: 0', 'PATIENT_x','PATIENT_y','ENCOUNTER', 'DESCRIPTION','Id'])
specific_columns = ['Arthritis Risk',
       'Alcohol/Drug Abuse', 'Air Quality', 'Lung Diseases', 'Heart Diseases',
       'Physical Health', 'Mental Health', 'Personality Disorder',
       'Housing Challenges', 'Transportation Risk', 'Social Associations']

for col in specific_columns:
    common_df[col] = (common_df[col] != 0).astype(int)
    
common_df.to_csv("/kaggle/working/test_data.csv")